# MNIST semirette (v5)

In [152]:
# load libraries
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow.keras as keraster
import tensorflow as tf
import keras
from   keras.datasets import mnist
from   tensorflow.keras.models import Sequential
from   tensorflow.keras.layers import Dense

In [164]:
# definisco costante passo
PASSO = 0.5
NUMERO_RETTE = 10

In [93]:
# carica dati mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [251]:
# definisco funzione 'plot_image'
def plot_image(img):
    plt.imshow(img, cmap = 'gray')
    plt.axis('off')

# definisci funzione 'show_image'
def show_image(img):
    plt.figure(figsize=(4,4))
    plot_image(img)
    plt.show(block = False)


# definisco funzione trova il centro dell'immagine
def find_center(img):
    # trova i pixel con valore maggiore di 0
    y,x = np.where(img > 0)
    # trova il centro dell'immagine
    center = (int(np.round(np.mean(x),0)), int(np.round(np.mean(y),0)))
    return center

# definisco funzione 'plot_center'
def plot_center(center,color):
    plt.scatter(center[0], center[1], c = color)

# definisco funzione 'generate_lines_from'
def generate_lines_from(point, img, m, passo):

    # with dell'immagine
    l = len(img)-1

    # rinomino point in c
    c = point

    # determino l'intercetta della retta
    q = c[1]-m*c[0]

    # creo due liste vuote
    
    x = np.zeros(np.round(28*10*PASSO).astype(int))
    y = np.zeros(np.round(28*10*PASSO).astype(int))

    # aggiungo il centro all'array x e y
    x[0] = c[0]
    y[0] = c[1]
    
    i=0

    # while y[-1] < 27 e contemporaneamente x[-1] < 27
    while (y[i] < l) and (x[i] <= l):
        i+=1
        x[i]=(x[i-1]+passo)
        y[i]=(m*x[i-1]+q)

    # se x[-1] > 27 o y[-1] > 27 allora elimina l'ultimo elemento di x e y
    if x[i] > l or y[i] > l:
        i-=1

    
    return x[0:i], y[0:i]


# definisco funzione 'get_pixel_values' che ritorna un array con i valori di grigio 
def get_pixel_values(img, x, y):
    
    # inizializza l'array a 0 casta i valori di pixel in interi
    l = len(x)
    values = np.zeros(l)
    print("a")
    for i in range(len(x)):
        print("a")
        kernel = img[np.round(max(x[i]-1,0)).astype(int):
                     np.round(min(x[i]+1,l-1)).astype(int),
                     np.round(max(0,y[i]-1)).astype(int):
                     np.round(min(y[i]+1,l-1)).astype(int)]       
        print("b")
    
        print(i,values.shape)
        values[i] = np.max(kernel)
        print(kernel.shape)
        print("c")
        
    return values.astype(int)


# definisco funzione per calcolare distanza tra due punti
def calcola_distanza(punto_inizio, punto_fine):
    distanza = np.sqrt((punto_inizio[0] - punto_fine[0])**2 + (punto_inizio[1] - punto_fine[1])**2)
    np.round(distanza, 2)
    return distanza

# definisco funzione 'find intersection'
def find_intersections(img, x, y, soglia_l, soglia_h, show_image_flag):
    numero_intersezioni=0
    distanza_ret=1.5*28
    
    if (show_image_flag == 1):
        # plotto
        plt.figure(figsize=(8,4))
        # set axis limits to 0, 28
        plt.xlim(0, 28)
        plt.ylim(0, 28)
        plt.subplot(1,2,1)
        # plotta x e y sull'immagine
        plt.imshow(img, cmap = 'gray')
        plt.plot(x, y, color = 'red')
        # plotta il centro dell'immagine
        plt.scatter(c[0], c[1], color = 'blue')
        plt.subplot(1,2,2)
        plt.plot(pixel)
        plt.show()
    
    l_p = len(pixel)
    index = 0
    while index < l_p:
        while (index < l_p) and (pixel[index]<soglia_l):
            index+=1
        if (index == l_p):
            break
            
        inizio = index
        index+=1
        #print(index)
        #print(pixel[index])
        while (index < l_p) and (pixel[index]>=soglia_h):
            index+=1
        numero_intersezioni+=1
        m=np.round((inizio + index -1)/2).astype(int)
        if (numero_intersezioni==1):
            punto_inizio = (x[m],y[m])
            distanza_ret = calcola_distanza(punto_inizio,c) 
            #print("distanza centro=",distanza_ret)       
    return numero_intersezioni,distanza_ret

# definisci funzione che plotta le intersezioni
def plot_intersections(coordinate_intersezioni, color):
    for i in range(len(coordinate_intersezioni)):
        plt.scatter(coordinate_intersezioni[i][0], coordinate_intersezioni[i][1], color = color)

In [244]:
# creo funzione che trasforma x_train
def transform_dataset(length,show_image_flag,data,coefficiente_angolare):
    
    #length = len(data)
    #length = 5000
    # inizializzo matrice vuota
    x_transformed = np.zeros((length,2))
    
    for i in range(length):
        print(i+1,"/",length)
        
        img = data[i]
        print("image")
        c = find_center(img)
        print("centro")
        
        x, y = generate_lines_from(c, img, coefficiente_angolare, PASSO)
        print("rette")
        
        print(x.shape,y.shape)
        
        pixel = get_pixel_values(img, x, y)
        print("pixels")
        
        x_transformed[i,0],x_transformed[i,1] = find_intersections(img,x,y,150,150,show_image_flag)
        #input("Press Enter to continue...")
    return x_transformed

In [252]:
TEST = transform_dataset(5000,0,x_train,0.6)

1 / 5000
image
centro
rette
(26,) (26,)
a
a
b
0 (26,)
(2, 2)
c
a
b
1 (26,)
(2, 2)
c
a
b
2 (26,)
(2, 2)
c
a
b
3 (26,)
(2, 2)
c
a
b
4 (26,)
(2, 2)
c
a
b
5 (26,)
(2, 2)
c
a
b
6 (26,)
(2, 2)
c
a
b
7 (26,)
(2, 2)
c
a
b
8 (26,)
(2, 2)
c
a
b
9 (26,)
(2, 2)
c
a
b
10 (26,)
(2, 2)
c
a
b
11 (26,)
(2, 2)
c
a
b
12 (26,)
(2, 2)
c
a
b
13 (26,)
(2, 2)
c
a
b
14 (26,)
(2, 2)
c
a
b
15 (26,)
(2, 2)
c
a
b
16 (26,)
(2, 2)
c
a
b
17 (26,)
(2, 2)
c
a
b
18 (26,)
(2, 2)
c
a
b
19 (26,)
(2, 2)
c
a
b
20 (26,)
(2, 2)
c
a
b
21 (26,)
(1, 2)
c
a
b
22 (26,)
(1, 2)
c
a
b
23 (26,)
(1, 2)
c
a
b
24 (26,)


ValueError: zero-size array to reduction operation maximum which has no identity

In [177]:
# crea funzione 'recognize_digit' per riconoscere il numero che crei il modello sopra
def recognize_digit(shape, n_epochs, n_categories, x_train, y_train, x_test, y_test, path):

    # rendo categoriche le variabili di output
    y_train = keras.utils.to_categorical(y_train, n_categories)
    y_test = keras.utils.to_categorical(y_test, n_categories)

    # normalizzo usando la funzione di keras
    # x_train = keras.utils.normalize(x_train, axis = 1)
    # x_test = keras.utils.normalize(x_test, axis = 1)

    # creo il modello
    model = Sequential()

    # flattening
    # model.add(keras.layers.Flatten(input_shape = (shape,shape)))

    # aggiungo layer + input
    model.add(Dense(units = 100, activation = 'relu', input_shape = (shape,)))

    # aggiungo uno strato nascosto
    model.add(Dense(units = 2, activation = 'relu'))

    # output layer
    model.add(Dense(units = 10, activation = 'softmax'))

    model.summary()

    # addestro il modello
    model.compile(loss = 'categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x_train, y_train, epochs = n_epochs, verbose = 1, validation_data = (x_test, y_test));

    # Salvataggio del modello
    model.save(path)

    # ritorna history
    return history

# crea funzione per visualizzare i risultati
def plot_results(history):
    # Plot training & validation accuracy values
    plt.figure(figsize=(5,5))
    plt.subplot(2,1,1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='best')

    # summarize history for loss
    plt.subplot(2,1,2)
    plt.subplots_adjust(hspace = 0.5)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='best')
    plt.show()

In [ ]:
x, y = generate_lines_from(c, img, 0.6, PASSO)



In [153]:
math.pi
n_steps = 10
for alpha in range(0,2*math.pi/n_steps,2*math.pi)
    

3.141592653589793

In [175]:
ms = np.zeros((NUMERO_RETTE))
ms.shape

(10,)

In [176]:

val=np.linspace(2*math.pi/NUMERO_RETTE, 2*math.pi, NUMERO_RETTE)
print(val)
ms = np.zeros((NUMERO_RETTE)) # coefficienti angolari
i=0
for alpha in val:
    ms[i] = math.tan(alpha)
    print(m)
    i+=1


[0.62831853 1.25663706 1.88495559 2.51327412 3.14159265 3.76991118
 4.39822972 5.02654825 5.65486678 6.28318531]
-2.4492935982947064e-16
-2.4492935982947064e-16
-2.4492935982947064e-16
-2.4492935982947064e-16
-2.4492935982947064e-16
-2.4492935982947064e-16
-2.4492935982947064e-16
-2.4492935982947064e-16
-2.4492935982947064e-16
-2.4492935982947064e-16


In [167]:
ms = math.tan(alpha)
ms

-2.4492935982947064e-16

In [182]:
lengt=5000

In [183]:
x_train_2 = np.zeros((length,NUMERO_RETTE*2))

for i in range(NUMERO_RETTE):
    x_train_2[2*i,(2*i)+1] = transform_dataset(5000,0,x_train,ms[i])

1 / 5000


ValueError: zero-size array to reduction operation maximum which has no identity

In [184]:
A = transform_dataset(5000,0,x_train,ms[0])
A.shape

1 / 5000


ValueError: zero-size array to reduction operation maximum which has no identity

In [112]:
# invoco funzione transform_dataset
x_train_2 = transform_dataset(5000,0,x_train)

0 / 5000
1 / 5000
2 / 5000
3 / 5000
4 / 5000
5 / 5000
6 / 5000
7 / 5000
8 / 5000
9 / 5000
10 / 5000
11 / 5000
12 / 5000
13 / 5000
14 / 5000
15 / 5000
16 / 5000
17 / 5000
18 / 5000
19 / 5000
20 / 5000
21 / 5000
22 / 5000
23 / 5000
24 / 5000
25 / 5000
26 / 5000
27 / 5000
28 / 5000
29 / 5000
30 / 5000
31 / 5000
32 / 5000
33 / 5000
34 / 5000
35 / 5000
36 / 5000
37 / 5000
38 / 5000
39 / 5000
40 / 5000
41 / 5000
42 / 5000
43 / 5000
44 / 5000
45 / 5000
46 / 5000
47 / 5000
48 / 5000
49 / 5000
50 / 5000
51 / 5000
52 / 5000
53 / 5000
54 / 5000
55 / 5000
56 / 5000
57 / 5000
58 / 5000
59 / 5000
60 / 5000
61 / 5000
62 / 5000
63 / 5000
64 / 5000
65 / 5000
66 / 5000
67 / 5000
68 / 5000
69 / 5000
70 / 5000
71 / 5000
72 / 5000
73 / 5000
74 / 5000
75 / 5000
76 / 5000
77 / 5000
78 / 5000
79 / 5000
80 / 5000
81 / 5000
82 / 5000
83 / 5000
84 / 5000
85 / 5000
86 / 5000
87 / 5000
88 / 5000
89 / 5000
90 / 5000
91 / 5000
92 / 5000
93 / 5000
94 / 5000
95 / 5000
96 / 5000
97 / 5000
98 / 5000
99 / 5000
100 / 5000

756 / 5000
757 / 5000
758 / 5000
759 / 5000
760 / 5000
761 / 5000
762 / 5000
763 / 5000
764 / 5000
765 / 5000
766 / 5000
767 / 5000
768 / 5000
769 / 5000
770 / 5000
771 / 5000
772 / 5000
773 / 5000
774 / 5000
775 / 5000
776 / 5000
777 / 5000
778 / 5000
779 / 5000
780 / 5000
781 / 5000
782 / 5000
783 / 5000
784 / 5000
785 / 5000
786 / 5000
787 / 5000
788 / 5000
789 / 5000
790 / 5000
791 / 5000
792 / 5000
793 / 5000
794 / 5000
795 / 5000
796 / 5000
797 / 5000
798 / 5000
799 / 5000
800 / 5000
801 / 5000
802 / 5000
803 / 5000
804 / 5000
805 / 5000
806 / 5000
807 / 5000
808 / 5000
809 / 5000
810 / 5000
811 / 5000
812 / 5000
813 / 5000
814 / 5000
815 / 5000
816 / 5000
817 / 5000
818 / 5000
819 / 5000
820 / 5000
821 / 5000
822 / 5000
823 / 5000
824 / 5000
825 / 5000
826 / 5000
827 / 5000
828 / 5000
829 / 5000
830 / 5000
831 / 5000
832 / 5000
833 / 5000
834 / 5000
835 / 5000
836 / 5000
837 / 5000
838 / 5000
839 / 5000
840 / 5000
841 / 5000
842 / 5000
843 / 5000
844 / 5000
845 / 5000
846 / 5000

1476 / 5000
1477 / 5000
1478 / 5000
1479 / 5000
1480 / 5000
1481 / 5000
1482 / 5000
1483 / 5000
1484 / 5000
1485 / 5000
1486 / 5000
1487 / 5000
1488 / 5000
1489 / 5000
1490 / 5000
1491 / 5000
1492 / 5000
1493 / 5000
1494 / 5000
1495 / 5000
1496 / 5000
1497 / 5000
1498 / 5000
1499 / 5000
1500 / 5000
1501 / 5000
1502 / 5000
1503 / 5000
1504 / 5000
1505 / 5000
1506 / 5000
1507 / 5000
1508 / 5000
1509 / 5000
1510 / 5000
1511 / 5000
1512 / 5000
1513 / 5000
1514 / 5000
1515 / 5000
1516 / 5000
1517 / 5000
1518 / 5000
1519 / 5000
1520 / 5000
1521 / 5000
1522 / 5000
1523 / 5000
1524 / 5000
1525 / 5000
1526 / 5000
1527 / 5000
1528 / 5000
1529 / 5000
1530 / 5000
1531 / 5000
1532 / 5000
1533 / 5000
1534 / 5000
1535 / 5000
1536 / 5000
1537 / 5000
1538 / 5000
1539 / 5000
1540 / 5000
1541 / 5000
1542 / 5000
1543 / 5000
1544 / 5000
1545 / 5000
1546 / 5000
1547 / 5000
1548 / 5000
1549 / 5000
1550 / 5000
1551 / 5000
1552 / 5000
1553 / 5000
1554 / 5000
1555 / 5000
1556 / 5000
1557 / 5000
1558 / 5000
1559

2171 / 5000
2172 / 5000
2173 / 5000
2174 / 5000
2175 / 5000
2176 / 5000
2177 / 5000
2178 / 5000
2179 / 5000
2180 / 5000
2181 / 5000
2182 / 5000
2183 / 5000
2184 / 5000
2185 / 5000
2186 / 5000
2187 / 5000
2188 / 5000
2189 / 5000
2190 / 5000
2191 / 5000
2192 / 5000
2193 / 5000
2194 / 5000
2195 / 5000
2196 / 5000
2197 / 5000
2198 / 5000
2199 / 5000
2200 / 5000
2201 / 5000
2202 / 5000
2203 / 5000
2204 / 5000
2205 / 5000
2206 / 5000
2207 / 5000
2208 / 5000
2209 / 5000
2210 / 5000
2211 / 5000
2212 / 5000
2213 / 5000
2214 / 5000
2215 / 5000
2216 / 5000
2217 / 5000
2218 / 5000
2219 / 5000
2220 / 5000
2221 / 5000
2222 / 5000
2223 / 5000
2224 / 5000
2225 / 5000
2226 / 5000
2227 / 5000
2228 / 5000
2229 / 5000
2230 / 5000
2231 / 5000
2232 / 5000
2233 / 5000
2234 / 5000
2235 / 5000
2236 / 5000
2237 / 5000
2238 / 5000
2239 / 5000
2240 / 5000
2241 / 5000
2242 / 5000
2243 / 5000
2244 / 5000
2245 / 5000
2246 / 5000
2247 / 5000
2248 / 5000
2249 / 5000
2250 / 5000
2251 / 5000
2252 / 5000
2253 / 5000
2254

2868 / 5000
2869 / 5000
2870 / 5000
2871 / 5000
2872 / 5000
2873 / 5000
2874 / 5000
2875 / 5000
2876 / 5000
2877 / 5000
2878 / 5000
2879 / 5000
2880 / 5000
2881 / 5000
2882 / 5000
2883 / 5000
2884 / 5000
2885 / 5000
2886 / 5000
2887 / 5000
2888 / 5000
2889 / 5000
2890 / 5000
2891 / 5000
2892 / 5000
2893 / 5000
2894 / 5000
2895 / 5000
2896 / 5000
2897 / 5000
2898 / 5000
2899 / 5000
2900 / 5000
2901 / 5000
2902 / 5000
2903 / 5000
2904 / 5000
2905 / 5000
2906 / 5000
2907 / 5000
2908 / 5000
2909 / 5000
2910 / 5000
2911 / 5000
2912 / 5000
2913 / 5000
2914 / 5000
2915 / 5000
2916 / 5000
2917 / 5000
2918 / 5000
2919 / 5000
2920 / 5000
2921 / 5000
2922 / 5000
2923 / 5000
2924 / 5000
2925 / 5000
2926 / 5000
2927 / 5000
2928 / 5000
2929 / 5000
2930 / 5000
2931 / 5000
2932 / 5000
2933 / 5000
2934 / 5000
2935 / 5000
2936 / 5000
2937 / 5000
2938 / 5000
2939 / 5000
2940 / 5000
2941 / 5000
2942 / 5000
2943 / 5000
2944 / 5000
2945 / 5000
2946 / 5000
2947 / 5000
2948 / 5000
2949 / 5000
2950 / 5000
2951

3553 / 5000
3554 / 5000
3555 / 5000
3556 / 5000
3557 / 5000
3558 / 5000
3559 / 5000
3560 / 5000
3561 / 5000
3562 / 5000
3563 / 5000
3564 / 5000
3565 / 5000
3566 / 5000
3567 / 5000
3568 / 5000
3569 / 5000
3570 / 5000
3571 / 5000
3572 / 5000
3573 / 5000
3574 / 5000
3575 / 5000
3576 / 5000
3577 / 5000
3578 / 5000
3579 / 5000
3580 / 5000
3581 / 5000
3582 / 5000
3583 / 5000
3584 / 5000
3585 / 5000
3586 / 5000
3587 / 5000
3588 / 5000
3589 / 5000
3590 / 5000
3591 / 5000
3592 / 5000
3593 / 5000
3594 / 5000
3595 / 5000
3596 / 5000
3597 / 5000
3598 / 5000
3599 / 5000
3600 / 5000
3601 / 5000
3602 / 5000
3603 / 5000
3604 / 5000
3605 / 5000
3606 / 5000
3607 / 5000
3608 / 5000
3609 / 5000
3610 / 5000
3611 / 5000
3612 / 5000
3613 / 5000
3614 / 5000
3615 / 5000
3616 / 5000
3617 / 5000
3618 / 5000
3619 / 5000
3620 / 5000
3621 / 5000
3622 / 5000
3623 / 5000
3624 / 5000
3625 / 5000
3626 / 5000
3627 / 5000
3628 / 5000
3629 / 5000
3630 / 5000
3631 / 5000
3632 / 5000
3633 / 5000
3634 / 5000
3635 / 5000
3636

4244 / 5000
4245 / 5000
4246 / 5000
4247 / 5000
4248 / 5000
4249 / 5000
4250 / 5000
4251 / 5000
4252 / 5000
4253 / 5000
4254 / 5000
4255 / 5000
4256 / 5000
4257 / 5000
4258 / 5000
4259 / 5000
4260 / 5000
4261 / 5000
4262 / 5000
4263 / 5000
4264 / 5000
4265 / 5000
4266 / 5000
4267 / 5000
4268 / 5000
4269 / 5000
4270 / 5000
4271 / 5000
4272 / 5000
4273 / 5000
4274 / 5000
4275 / 5000
4276 / 5000
4277 / 5000
4278 / 5000
4279 / 5000
4280 / 5000
4281 / 5000
4282 / 5000
4283 / 5000
4284 / 5000
4285 / 5000
4286 / 5000
4287 / 5000
4288 / 5000
4289 / 5000
4290 / 5000
4291 / 5000
4292 / 5000
4293 / 5000
4294 / 5000
4295 / 5000
4296 / 5000
4297 / 5000
4298 / 5000
4299 / 5000
4300 / 5000
4301 / 5000
4302 / 5000
4303 / 5000
4304 / 5000
4305 / 5000
4306 / 5000
4307 / 5000
4308 / 5000
4309 / 5000
4310 / 5000
4311 / 5000
4312 / 5000
4313 / 5000
4314 / 5000
4315 / 5000
4316 / 5000
4317 / 5000
4318 / 5000
4319 / 5000
4320 / 5000
4321 / 5000
4322 / 5000
4323 / 5000
4324 / 5000
4325 / 5000
4326 / 5000
4327

4935 / 5000
4936 / 5000
4937 / 5000
4938 / 5000
4939 / 5000
4940 / 5000
4941 / 5000
4942 / 5000
4943 / 5000
4944 / 5000
4945 / 5000
4946 / 5000
4947 / 5000
4948 / 5000
4949 / 5000
4950 / 5000
4951 / 5000
4952 / 5000
4953 / 5000
4954 / 5000
4955 / 5000
4956 / 5000
4957 / 5000
4958 / 5000
4959 / 5000
4960 / 5000
4961 / 5000
4962 / 5000
4963 / 5000
4964 / 5000
4965 / 5000
4966 / 5000
4967 / 5000
4968 / 5000
4969 / 5000
4970 / 5000
4971 / 5000
4972 / 5000
4973 / 5000
4974 / 5000
4975 / 5000
4976 / 5000
4977 / 5000
4978 / 5000
4979 / 5000
4980 / 5000
4981 / 5000
4982 / 5000
4983 / 5000
4984 / 5000
4985 / 5000
4986 / 5000
4987 / 5000
4988 / 5000
4989 / 5000
4990 / 5000
4991 / 5000
4992 / 5000
4993 / 5000
4994 / 5000
4995 / 5000
4996 / 5000
4997 / 5000
4998 / 5000
4999 / 5000


In [115]:
x_test_2 = transform_dataset(1000,0,x_test)

0 / 1000
1 / 1000
2 / 1000
3 / 1000
4 / 1000
5 / 1000
6 / 1000
7 / 1000
8 / 1000
9 / 1000
10 / 1000
11 / 1000
12 / 1000
13 / 1000
14 / 1000
15 / 1000
16 / 1000
17 / 1000
18 / 1000
19 / 1000
20 / 1000
21 / 1000
22 / 1000
23 / 1000
24 / 1000
25 / 1000
26 / 1000
27 / 1000
28 / 1000
29 / 1000
30 / 1000
31 / 1000
32 / 1000
33 / 1000
34 / 1000
35 / 1000
36 / 1000
37 / 1000
38 / 1000
39 / 1000
40 / 1000
41 / 1000
42 / 1000
43 / 1000
44 / 1000
45 / 1000
46 / 1000
47 / 1000
48 / 1000
49 / 1000
50 / 1000
51 / 1000
52 / 1000
53 / 1000
54 / 1000
55 / 1000
56 / 1000
57 / 1000
58 / 1000
59 / 1000
60 / 1000
61 / 1000
62 / 1000
63 / 1000
64 / 1000
65 / 1000
66 / 1000
67 / 1000
68 / 1000
69 / 1000
70 / 1000
71 / 1000
72 / 1000
73 / 1000
74 / 1000
75 / 1000
76 / 1000
77 / 1000
78 / 1000
79 / 1000
80 / 1000
81 / 1000
82 / 1000
83 / 1000
84 / 1000
85 / 1000
86 / 1000
87 / 1000
88 / 1000
89 / 1000
90 / 1000
91 / 1000
92 / 1000
93 / 1000
94 / 1000
95 / 1000
96 / 1000
97 / 1000
98 / 1000
99 / 1000
100 / 1000

773 / 1000
774 / 1000
775 / 1000
776 / 1000
777 / 1000
778 / 1000
779 / 1000
780 / 1000
781 / 1000
782 / 1000
783 / 1000
784 / 1000
785 / 1000
786 / 1000
787 / 1000
788 / 1000
789 / 1000
790 / 1000
791 / 1000
792 / 1000
793 / 1000
794 / 1000
795 / 1000
796 / 1000
797 / 1000
798 / 1000
799 / 1000
800 / 1000
801 / 1000
802 / 1000
803 / 1000
804 / 1000
805 / 1000
806 / 1000
807 / 1000
808 / 1000
809 / 1000
810 / 1000
811 / 1000
812 / 1000
813 / 1000
814 / 1000
815 / 1000
816 / 1000
817 / 1000
818 / 1000
819 / 1000
820 / 1000
821 / 1000
822 / 1000
823 / 1000
824 / 1000
825 / 1000
826 / 1000
827 / 1000
828 / 1000
829 / 1000
830 / 1000
831 / 1000
832 / 1000
833 / 1000
834 / 1000
835 / 1000
836 / 1000
837 / 1000
838 / 1000
839 / 1000
840 / 1000
841 / 1000
842 / 1000
843 / 1000
844 / 1000
845 / 1000
846 / 1000
847 / 1000
848 / 1000
849 / 1000
850 / 1000
851 / 1000
852 / 1000
853 / 1000
854 / 1000
855 / 1000
856 / 1000
857 / 1000
858 / 1000
859 / 1000
860 / 1000
861 / 1000
862 / 1000
863 / 1000

In [140]:
y_train = y_train[0:5000]
y_test = y_train[0:1000]

In [150]:
# invoco funzione recognize_digit per creare modello
history = recognize_digit(2, 100, 10, x_train_2, y_train, x_test_2, y_test, "4_mnist/models/np.mnist.model.h6")

# invoco funzione per visualizzare i risultati
plot_results(history)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 100)               300       
                                                                 
 dense_19 (Dense)            (None, 2)                 202       
                                                                 
 dense_20 (Dense)            (None, 10)                30        
                                                                 
Total params: 532
Trainable params: 532
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
157/157 [==============================] - 1s 3ms/step - loss: 2.3021 - accuracy: 0.1102 - val_loss: 2.3011 - val_accuracy: 0.1160
Epoch 2/100
157/157 [==============================] - 0s 3ms/step - loss: 2.3011 - accuracy: 0.1126 - val_loss: 2.3003 - val_accuracy: 0.1160
Epoch 3/100
157/157 [=============

157/157 [==============================] - 0s 2ms/step - loss: 2.3000 - accuracy: 0.1126 - val_loss: 2.2992 - val_accuracy: 0.1160
Epoch 53/100
143/157 [==========================>...] - ETA: 0s - loss: 2.3003 - accuracy: 0.1121

KeyboardInterrupt: 